In [133]:
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
from datetime import datetime

pd.options.plotting.backend = "holoviews"


In [2]:
cred = {
    "host": "localhost",
    "dbname": "propeller_ads",
    "user": "glebsokolov",
    "password": "",
}
from sqlalchemy import create_engine

con = create_engine(
    f'postgresql://{cred["user"]}:{cred["password"]}@{cred["host"]}/{cred["dbname"]}'
)


def select(sql):
    return pd.read_sql(sql, con)


In [149]:
def number_of_employees(rng):
    try:
        if "2022" in rng:
            return 10
        elif "+" in rng:
            return 10001
        else:
            dash = rng.index("-")
            return int(rng[dash + 1 :])
    except:
        return 0


In [159]:
table0 = pd.read_excel(
    "/Users/glebsokolov/Downloads/test task raw data .xlsx", sheet_name=0
)
table1 = pd.read_excel(
    "/Users/glebsokolov/Downloads/test task raw data .xlsx", sheet_name=1
)
table1["id"] = table1.index
table2 = pd.read_excel(
    "/Users/glebsokolov/Downloads/test task raw data .xlsx", sheet_name=2
)
table2["id"] = table2.index
table2["max_employees"] = table2["Number of Employees"].apply(number_of_employees)
table3 = pd.read_excel(
    "/Users/glebsokolov/Downloads/test task raw data .xlsx", sheet_name=3
)
table3["id"] = table3.index
table4 = pd.read_excel(
    "/Users/glebsokolov/Downloads/test task raw data .xlsx", sheet_name=4
)
table4["id"] = table4.index
table5 = pd.read_excel(
    "/Users/glebsokolov/Downloads/test task raw data .xlsx", sheet_name=5
)
table5["id"] = table5.index
table5 = table5.rename(columns={"Headquarters Location": "hq_loc"})
table6 = pd.read_excel(
    "/Users/glebsokolov/Downloads/test task raw data .xlsx", sheet_name=6
)
table6["id"] = table6.index
table7 = pd.read_excel(
    "/Users/glebsokolov/Downloads/test task raw data .xlsx", sheet_name=7
)
table7["id"] = table7.index


/var/folders/t0/fw3jvl196_v78v94hzhxhpgw0000gn/T/ipykernel_36301/1361070563.py:14: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  table3 = pd.read_excel(


In [ ]:
sql = """WITH inter AS
  (SELECT *
   FROM
     (SELECT *,
             (DENSE_RANK() OVER (
                                 ORDER BY max_emp::float) * "Monthly Visits"::float + "Total Funding Amount"::float) AS first_score,
             DENSE_RANK() OVER (
                                ORDER BY max_emp::float) AS "Number of Employees"
      FROM
        (SELECT a."Organization Name",
                a."id",
                b."Website", --c."Number of Employees",
CASE WHEN c."Number of Employees" IS NULL THEN \'0\' ELSE c."Number of Employees" END as "N of Employees",
          d."Founded Date",
          d."Industries",
          --e."Monthly Visits",
          CASE
            WHEN e."Monthly Visits" is NULL THEN \'0\' ELSE e."Monthly Visits" END as "Monthly Visits",
          f."hq_loc",
          --g."Total Funding Amount",
          CASE
            WHEN g."Total Funding Amount" is NULL THEN \'0\' ELSE g."Total Funding Amount" END as "Total Funding Amount",
          CASE
            WHEN c."Number of Employees" IS NULL THEN \'0\'
            WHEN c."Number of Employees" LIKE \'%%+%%\' THEN \'10001\'
            WHEN c."Number of Employees" LIKE \'%%10-01%%\' THEN \'10\'
            ELSE substring(c.\"Number of Employees\" from position('-' in c.\"Number of Employees\")+1 for length(c.\"Number of Employees\"))
        END as max_emp, -- обработаем строку с числом сотрудников, оставив правую границу диапазона ()
       CASE
           WHEN g."Total Funding Amount" IS NULL
                AND e."Monthly Visits" IS NULL THEN 0
           WHEN e."Monthly Visits" IS NULL THEN 0
           WHEN g."Total Funding Amount" IS NULL THEN 0
       END AS "Funding per Visitor", -- дополнительный сортировочный критерий
       h."had cooperation with us"
FROM id_name a
LEFT JOIN name_site b ON a.id = b.index
LEFT JOIN employees c ON a.id=c.id
LEFT JOIN company_info d ON a.id=d.id
LEFT JOIN monthly_visits e ON a.id=e.id
LEFT JOIN hq f ON a.id=f.id
LEFT JOIN funding g ON a.id=g.id
LEFT JOIN cooperation h ON a.id=h.id
-- в каждую из таблиц была добавлена колонка id соответсвуюшая номеру строки
WHERE f.hq_loc SIMILAR TO '%%United Kingdom%%|%%Portugal%%|%%Poland%%') t
      ORDER BY CASE
                   WHEN t."Monthly Visits" IS NULL THEN 1
                   ELSE 0
               END, -- чтобы пропущенные значения не были вверху таблицы
 t."Monthly Visits" DESC, max_emp::float DESC, CASE
                                                   WHEN t."Total Funding Amount" IS NULL THEN 1
                                                   ELSE 0
                                               END,
                                               t."Total Funding Amount" DESC) tt),
     TEMP AS
  (SELECT (first_score-min(first_score))/(max(inter.first_score) - min(inter.first_score))
   FROM inter
   GROUP BY first_score)
SELECT inter.*
FROM inter
"""
res = select(sql)
res["Age"] = round(
    (pd.to_datetime(datetime.now()) - res["Founded Date"]) / np.timedelta64(1, "Y")
)
normalize = lambda col: (res[col] - res[col].min()) / (res[col].max() - res[col].min())
res["Age"] = normalize("Age")
# вычислим дополнительно две метрики, первая будет являться  min-max масштабированной суммой первых двух колонок (monthly visits, funding amount), умноженной на коэффициент, отражающий количество сотрудников
# вторая метрика  - сумма нормализованных (monthly visits, funding amount, age) и коэффициента-количества сотрудников
# в числовых колонках null заменены на 0 для удобства, поэтому метрики будут вычисляться корректно только для строк, где нет пропусков
res["first_score"] = res["Number of Employees"] * res["first_score"].apply(
    lambda x: (x - res["first_score"].min()) / res["first_score"].max()
    - res["first_score"].min()
)
res["second_score"] = res["Number of Employees"] + (
    res["Monthly Visits"] / res["Monthly Visits"].max()
    + res["Total Funding Amount"] / res["Total Funding Amount"].max()
    + res["Age"] / res["Age"].max()
)
cols = [
    "Organization Name",
    "Industries",
    "Monthly Visits",
    "Total Funding Amount",
    "Funding per Visitor",
    "had cooperation with us",
    "N of Employees",
    "first_score",
    "second_score",
]
res["N of Employees"] = res["N of Employees"].apply(
    lambda x: "<10" if "2022" in x else x
)
# в итоговой таблице необходимо только определить приоритетность сортировки для метрик
res[cols]


,Organization Name,Industries,Monthly Visits,Total Funding Amount,Funding per Visitor,had cooperation with us,N of Employees,first_score,second_score
0,Bet365,"Gambling, Leisure",113813930.0,0.0,0.0,None,5001-10000,9.000000,10.148148
1,Sky Betting and Gaming,"eSports, Gambling",14515092.0,0.0,0.0,None,1001-5000,0.906905,8.275682
2,Betfred,"Casino, Gambling",3979209.0,0.0,0.0,None,10001+,0.388471,10.434962
3,Virtue Fusion,"Gambling, Gaming, Media and Entertainment, Software",3073090.0,0.0,0.0,None,11-50,0.027001,3.189964
4,Replay Poker,Gambling,2347338.0,0.0,0.0,None,<10,0.009166,2.139143
...,...,...,...,...,...,...,...,...,...
162,Friendbet,"Gambling, Soccer, Social Media, Sports",0.0,0.0,0.0,yes,0,0.000000,1.081481
163,Tote Sport,"Gambling, Recreation, Sports",0.0,0.0,0.0,yes,0,0.000000,1.688889
164,myLotto24 Ltd.,"Casino, Gambling",0.0,0.0,0.0,yes,0,0.000000,NaN
165,Bluesq,"eSports, Gambling, Information Technology, Internet of Things, Messaging, Sports",0.0,0.0,0.0,yes,0,0.000000,1.162963
